In [3]:
import pandas as pd

sectoral_tfp_df_file_path = '../raw_data/productivity_data/major-industry-total-factor-productivity-klems.xlsx'
sectoral_tfp_df = pd.read_excel(sectoral_tfp_df_file_path, header=2)

bds_3_df_file_path = '../raw_data/dynamism_data/bds2022_vcn3.csv'
bds_3_df = pd.read_csv(bds_3_df_file_path)

In [4]:
sectoral_tfp_df['NAICS'].unique()

array(['11', '111-112', '113-115', '21', '211', '212', '213', '22', '23',
       'MN', 'DM', 'ND', '321', '327', '331', '332', '333', '334', '335',
       '3361-3363', '3364-3369', '337', '339', '311-312', '313-314',
       '315-316', '322', '323', '324', '325', '326', '42,44-45', '42',
       '44,45', '48-49', '481', '482', '483', '484', '485', '486',
       '487,488,492', '493', '51', '511', '512', '515,517', '518-519',
       '52-53', '52', '521-522', '523', '524', '525', '53', '531',
       '532-533', '54-81', '54-56', '54', '5411', '5415',
       '5412-5414,5416-5419', '55', '56', '561', '562', '61-62', '61',
       '62', '621', '622-623', '624', '71-72', '71', '711-712', '713',
       '72', '721', '722', '81'], dtype=object)

In [5]:
sectoral_tfp_df['NAICS'] = sectoral_tfp_df['NAICS'].replace({
    'MN': '31-33',  
    'DM': '33',     
    'ND': '31'      
})

In [6]:
sectoral_tfp_df['NAICS'].unique()

array(['11', '111-112', '113-115', '21', '211', '212', '213', '22', '23',
       '31-33', '33', '31', '321', '327', '331', '332', '333', '334',
       '335', '3361-3363', '3364-3369', '337', '339', '311-312',
       '313-314', '315-316', '322', '323', '324', '325', '326',
       '42,44-45', '42', '44,45', '48-49', '481', '482', '483', '484',
       '485', '486', '487,488,492', '493', '51', '511', '512', '515,517',
       '518-519', '52-53', '52', '521-522', '523', '524', '525', '53',
       '531', '532-533', '54-81', '54-56', '54', '5411', '5415',
       '5412-5414,5416-5419', '55', '56', '561', '562', '61-62', '61',
       '62', '621', '622-623', '624', '71-72', '71', '711-712', '713',
       '72', '721', '722', '81'], dtype=object)

In [7]:
three_digit_sectoral_df = sectoral_tfp_df[sectoral_tfp_df['NAICS'].str.len() == 3].copy()

print(sorted(three_digit_sectoral_df['NAICS'].unique()))

['211', '212', '213', '321', '322', '323', '324', '325', '326', '327', '331', '332', '333', '334', '335', '337', '339', '481', '482', '483', '484', '485', '486', '493', '511', '512', '523', '524', '525', '531', '561', '562', '621', '624', '713', '721', '722']


In [8]:
long_df = three_digit_sectoral_df.melt(
    id_vars=['NAICS', 'Industry', 'Basis', 'Measure', 'Units'], 
    var_name='year', 
    value_name='value' 
)

In [9]:
tfp_long = long_df[long_df['Measure'] == 'Total factor productivity'].copy()
output_long = long_df[long_df['Measure'] == 'Sectoral output'].copy()

In [10]:
output_pivot = output_long.pivot_table(
    values='value', 
    index='year', 
    columns='NAICS', 
    aggfunc='sum',
    fill_value=0
)

weights = (
    output_pivot.apply(lambda row: row / (row.sum() + 1e-10), axis=1)
    .stack()
    .reset_index()
    .rename(columns={0: 'weight'})
)

# Merge weights with TFP data
tfp_with_weights = tfp_long.merge(
    weights,
    on=['year', 'NAICS'],
    how='left'
).fillna({'weight': 0})

C:\Users\Lamb\AppData\Local\Temp\ipykernel_25572\4150187125.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output_pivot = output_long.pivot_table(


In [11]:
tfp_units = tfp_with_weights['Units'].value_counts()
print("Units used for Total factor productivity:\n", tfp_units)

tfp_pivoted = tfp_with_weights.pivot(
    index=['NAICS', 'Industry', 'Basis', 'year', 'weight'],
    columns='Units',
    values='value'
).reset_index()

tfp_pivoted.columns.name = None

tfp_pivoted = tfp_pivoted.rename(columns={
    'Index (2017=100)': 'tfp_index_2017',
    '% Change from previous year': 'tfp_pct_change'
})

Units used for Total factor productivity:
 Units
Index (2017=100)               1369
% Change from previous year    1369
Name: count, dtype: int64


In [12]:
bds_3_df = bds_3_df.rename(columns={'vcnaics3': 'NAICS'})

In [13]:
bds_3_df['year'] = bds_3_df['year'].astype(int)
bds_87_on = bds_3_df[bds_3_df['year'] >= 1987]

In [14]:
print("bds_87_on NAICS unique values:", bds_87_on['NAICS'].unique()[:5])
print("tfp_pivoted NAICS unique values:", tfp_pivoted['NAICS'].unique()[:5])

bds_87_on NAICS unique values: [113 114 115 211 212]
tfp_pivoted NAICS unique values: ['211' '212' '213' '321' '322']


In [15]:
print("bds_87_on columns:", bds_87_on.columns.tolist())
print("tfp_pivoted columns:", tfp_pivoted.columns.tolist())

bds_87_on columns: ['year', 'NAICS', 'firms', 'estabs', 'emp', 'denom', 'estabs_entry', 'estabs_entry_rate', 'estabs_exit', 'estabs_exit_rate', 'job_creation', 'job_creation_births', 'job_creation_continuers', 'job_creation_rate_births', 'job_creation_rate', 'job_destruction', 'job_destruction_deaths', 'job_destruction_continuers', 'job_destruction_rate_deaths', 'job_destruction_rate', 'net_job_creation', 'net_job_creation_rate', 'reallocation_rate', 'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp']
tfp_pivoted columns: ['NAICS', 'Industry', 'Basis', 'year', 'weight', 'tfp_pct_change', 'tfp_index_2017']


In [16]:
bds_87_on = bds_87_on.copy()
bds_87_on['NAICS'] = bds_87_on['NAICS'].astype(str)
tfp_pivoted['NAICS'] = tfp_pivoted['NAICS'].astype(str)

In [17]:
tfp_pivoted['year'] = tfp_pivoted['year'].astype(int)

# 6) Now merge
merged = pd.merge(
    bds_87_on,
    tfp_pivoted, 
    on=['NAICS','year'],   
    how='inner'
)

In [18]:
object_columns = [
    'estabs_entry', 'estabs_entry_rate', 'estabs_exit', 'estabs_exit_rate',
    'job_creation_births', 'job_creation_rate_births', 'job_destruction_deaths',
    'job_destruction_rate_deaths', 'firmdeath_estabs',
    'tfp_pct_change', 'tfp_index_2017'
]

for col in object_columns:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [20]:
merged['firmdeath_firms'] = pd.to_numeric(merged['firmdeath_firms'], errors='coerce')
merged['firms'] = pd.to_numeric(merged['firms'], errors='coerce')
merged['firms_percent_destroyed'] = (merged['firmdeath_firms'] / merged['firms']) * 100

In [21]:
output_path = '../processed_data/three_digit_NAICS_bds_tfp.csv'
merged.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

Data saved to ../processed_data/three_digit_NAICS_bds_tfp.csv
